In [1]:
import optuna
from surprise import KNNBasic,KNNWithMeans,KNNWithZScore,KNNBaseline, Dataset, Reader
from surprise.model_selection import cross_validate
import numpy as np
np.random.seed(42)
# Define the dataset path and the reader format
file_path = 'u.data'  # Make sure to provide the correct path to your dataset
reader = Reader(line_format='user item rating timestamp', sep='\t')

# Load the dataset
data = Dataset.load_from_file(file_path, reader=reader)

In [2]:
# Define the objective function for hyperparameter optimization for KNNBasic
def knn_basic_objective(trial):
    # Hyperparameters to optimize
    k = trial.suggest_int('k', 20, 100)
    min_k = trial.suggest_int('min_k', 1, 10)
    sim_options = {
        'name': trial.suggest_categorical('sim_options_name', ['msd', 'cosine', 'pearson', 'pearson_baseline']),
        'user_based': trial.suggest_categorical('user_based', [False, True]),
        'min_support': trial.suggest_int('min_support', 1, 5),
    }
    # Additional parameters for pearson_baseline
    if sim_options['name'] == 'pearson_baseline':
        sim_options['shrinkage'] = trial.suggest_int('shrinkage', 0, 200)
    bsl_options = {}
    if sim_options['name'] == 'pearson_baseline':
            # Baseline options are used only with pearson_baseline similarity
        bsl_method = trial.suggest_categorical('bsl_method', ['als', 'sgd'])
        bsl_options['method'] = bsl_method
        bsl_options['n_epochs'] = trial.suggest_int('bsl_n_epochs', 10, 50)  # Default is 10
        if bsl_method == 'als':
            bsl_options['reg_u'] = trial.suggest_float('als_reg_u', 1, 20)  # Default is 15
            bsl_options['reg_i'] = trial.suggest_float('als_reg_i', 1, 20)  # Default is 10
        elif bsl_method == 'sgd':
            bsl_options['reg'] = trial.suggest_float('sgd_reg', 0.01, 0.2)  # Default is 0.02
            bsl_options['learning_rate'] = trial.suggest_float('sgd_learning_rate', 0.001, 0.01)  # Default is 0.005
    # Instantiate the KNNBasic algorithm with the suggested parameters
    algo = KNNBasic(k=k, min_k=min_k, sim_options=sim_options,bsl_options=bsl_options if bsl_options else None)

    # Perform cross-validation and return the mean RMSE
    # Perform cross-validation and return the mean RMSE, MAE, and FCP
    cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE', 'FCP'], cv=5, verbose=False)
    mean_rmse = np.mean(cv_results['test_rmse'])
    mean_mae = np.mean(cv_results['test_mae'])
    mean_fcp = np.mean(cv_results['test_fcp'])

    return mean_rmse, mean_mae, mean_fcp

# Create a multi-objective study object and optimize the objective function
study = optuna.create_study(directions=['minimize', 'minimize', 'maximize'], sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(knn_basic_objective, n_trials=100,n_jobs=-1)  # Number of trials can be adjusted

# Output the Pareto-optimal solutions
pareto_front = study.best_trials

# Save the Pareto-optimal solutions and their parameters to a file
with open('pareto_front_trials_and_params_KNNBasic.txt', 'w') as f:
    for trial in pareto_front:
        f.write(f"KNNBasic")
        f.write(f"Trial#{trial.number}\n")
        f.write(f"RMSE: {trial.values[0]}\n")
        f.write(f"MAE: {trial.values[1]}\n")
        f.write(f"FCP: {trial.values[2]}\n")
        f.write(f"Parameters: {trial.params}\n\n")

[I 2023-11-10 17:47:01,740] A new study created in memory with name: no-name-c9556566-f6c0-4bc0-a87e-d6f82283c953


Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Computing the pearson similarity matrix...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Estimating biases using sgd...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the c

[I 2023-11-10 17:51:12,911] Trial 1 finished with values: [0.9764915625156764, 0.7699002678479898, 0.7075427348969048] and parameters: {'k': 21, 'min_k': 1, 'sim_options_name': 'msd', 'user_based': True, 'min_support': 1}. 


Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...


[I 2023-11-10 17:51:24,540] Trial 8 finished with values: [0.9851214867928203, 0.7792079880744317, 0.7034742874957773] and parameters: {'k': 58, 'min_k': 7, 'sim_options_name': 'msd', 'user_based': True, 'min_support': 3}. 


...

[I 2023-11-10 18:19:38,309] Trial 93 finished with values: [0.9915285931173953, 0.7856596081211775, 0.7021596908901836] and parameters: {'k': 88, 'min_k': 6, 'sim_options_name': 'msd', 'user_based': True, 'min_support': 3}. 
[I 2023-11-10 18:19:38,594] Trial 94 finished with values: [0.9844485283692241, 0.7790405982109228, 0.7072707240407481] and parameters: {'k': 60, 'min_k': 5, 'sim_options_name': 'msd', 'user_based': True, 'min_support': 3}. 


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...


[I 2023-11-10 18:19:53,890] Trial 95 finished with values: [0.9836649335658263, 0.7783304463599959, 0.7047468642895139] and parameters: {'k': 58, 'min_k': 5, 'sim_options_name': 'msd', 'user_based': True, 'min_support': 3}. 


Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.Done computing similarity matrix.

Computing the msd similarity matrix...
Done computing similarity matrix.


[I 2023-11-10 18:20:08,571] Trial 99 finished with values: [0.9742005874858319, 0.7694753928207815, 0.705004340219681] and parameters: {'k': 27, 'min_k': 4, 'sim_options_name': 'msd', 'user_based': True, 'min_support': 3}. 
[I 2023-11-10 18:20:08,885] Trial 96 finished with values: [0.9935673805624201, 0.7869238780449314, 0.6854664437344182] and parameters: {'k': 98, 'min_k': 4, 'sim_options_name': 'msd', 'user_based': False, 'min_support': 3}. 
[I 2023-11-10 18:20:10,366] Trial 97 finished with values: [0.991897228281926, 0.7851332206406131, 0.685645338263976] and parameters: {'k': 93, 'min_k': 4, 'sim_options_name': 'msd', 'user_based': False, 'min_support': 3}. 
[I 2023-11-10 18:20:10,532] Trial 98 finished with values: [0.973251565268559, 0.7668547644833147, 0.6786111988665203] and parameters: {'k': 37, 'min_k': 4, 'sim_options_name': 'msd', 'user_based': False, 'min_support': 3}. 


In [3]:
# Define the objective function for hyperparameter optimization for KNNWithMeans
def knn_with_means_objective(trial):
    # Hyperparameters to optimize
    k = trial.suggest_int('k', 20, 100)
    min_k = trial.suggest_int('min_k', 1, 10)
    sim_options = {
        'name': trial.suggest_categorical('sim_options_name', ['msd', 'cosine', 'pearson', 'pearson_baseline']),
        'user_based': trial.suggest_categorical('user_based', [False, True]),
        'min_support': trial.suggest_int('min_support', 1, 5),
    }
    # Additional parameters for pearson_baseline
    if sim_options['name'] == 'pearson_baseline':
        sim_options['shrinkage'] = trial.suggest_int('shrinkage', 0, 200)
    bsl_options = {}
    if sim_options['name'] == 'pearson_baseline':
            # Baseline options are used only with pearson_baseline similarity
        bsl_method = trial.suggest_categorical('bsl_method', ['als', 'sgd'])
        bsl_options['method'] = bsl_method
        bsl_options['n_epochs'] = trial.suggest_int('bsl_n_epochs', 10, 50)  # Default is 10
        if bsl_method == 'als':
            bsl_options['reg_u'] = trial.suggest_float('als_reg_u', 1, 20)  # Default is 15
            bsl_options['reg_i'] = trial.suggest_float('als_reg_i', 1, 20)  # Default is 10
        elif bsl_method == 'sgd':
            bsl_options['reg'] = trial.suggest_float('sgd_reg', 0.01, 0.2)  # Default is 0.02
            bsl_options['learning_rate'] = trial.suggest_float('sgd_learning_rate', 0.001, 0.01)  # Default is 0.005
    # Instantiate the KNNBasic algorithm with the suggested parameters
    algo = KNNWithMeans(k=k, min_k=min_k, sim_options=sim_options,bsl_options=bsl_options if bsl_options else None)

    # Perform cross-validation and return the mean RMSE
    # Perform cross-validation and return the mean RMSE, MAE, and FCP
    cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE', 'FCP'], cv=5, verbose=False)
    mean_rmse = np.mean(cv_results['test_rmse'])
    mean_mae = np.mean(cv_results['test_mae'])
    mean_fcp = np.mean(cv_results['test_fcp'])

    return mean_rmse, mean_mae, mean_fcp

# Create a multi-objective study object and optimize the objective function
study = optuna.create_study(directions=['minimize', 'minimize', 'maximize'], sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(knn_with_means_objective, n_trials=100,n_jobs=-1)  # Number of trials can be adjusted

# Output the Pareto-optimal solutions
pareto_front = study.best_trials

# Save the Pareto-optimal solutions and their parameters to a file
with open('pareto_front_trials_and_params_KNNWithMeans.txt', 'w') as f:
    for trial in pareto_front:
        f.write(f"KNNWithMeans")
        f.write(f"Trial#{trial.number}\n")
        f.write(f"RMSE: {trial.values[0]}\n")
        f.write(f"MAE: {trial.values[1]}\n")
        f.write(f"FCP: {trial.values[2]}\n")
        f.write(f"Parameters: {trial.params}\n\n")

[I 2023-11-10 18:20:10,570] A new study created in memory with name: no-name-736b5053-2924-425b-aec8-6c38d51b8293


Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Estimating biases using sgd...
Done computing similarity matrix.
Estimating biases usingDone computing similarity matrix.
 sgd...
Computing the pearson similarity matrix...
Computing the cosine similarity matrix...
Computing the pearson similarity matrix...
Computing the cosine similarity matrix...
Computing the msd similarity matrix...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Estimating biases using als...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Estimating biases using als...
Estimating biases using als...
Computing the pearson similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matr

[I 2023-11-10 18:24:47,025] Trial 4 finished with values: [0.9436508378140538, 0.7341765914767315, 0.6951228454937294] and parameters: {'k': 24, 'min_k': 10, 'sim_options_name': 'pearson_baseline', 'user_based': True, 'min_support': 1, 'shrinkage': 107, 'bsl_method': 'sgd', 'bsl_n_epochs': 11, 'sgd_reg': 0.12116938248209691, 'sgd_learning_rate': 0.0036022869864400786}. 


Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.


[I 2023-11-10 18:25:03,638] Trial 1 finished with values: [0.9471151445098608, 0.7423852859136405, 0.7061100132723062] and parameters: {'k': 70, 'min_k': 5, 'sim_options_name': 'pearson', 'user_based': True, 'min_support': 1}. 


...

[I 2023-11-10 18:58:00,477] Trial 97 finished with values: [0.9258657989829935, 0.7235779278371178, 0.718784285632166] and parameters: {'k': 42, 'min_k': 3, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 5, 'shrinkage': 119, 'bsl_method': 'sgd', 'bsl_n_epochs': 37, 'sgd_reg': 0.1945858275339003, 'sgd_learning_rate': 0.006648705568823377}. 


Computing the pearson_baseline similarity matrix...


[I 2023-11-10 18:58:02,664] Trial 95 finished with values: [0.9287454320991488, 0.7258870999009361, 0.7152724424324609] and parameters: {'k': 100, 'min_k': 2, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 5, 'shrinkage': 119, 'bsl_method': 'sgd', 'bsl_n_epochs': 37, 'sgd_reg': 0.1996183891995122, 'sgd_learning_rate': 0.0070531822164997945}. 


Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


[I 2023-11-10 18:58:08,123] Trial 98 finished with values: [0.9255997692905341, 0.724122956749093, 0.7186402502507334] and parameters: {'k': 70, 'min_k': 3, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 5, 'shrinkage': 122, 'bsl_method': 'sgd', 'bsl_n_epochs': 37, 'sgd_reg': 0.18286505297800093, 'sgd_learning_rate': 0.006407743476523558}. 
[I 2023-11-10 18:58:12,767] Trial 99 finished with values: [0.926751543757278, 0.7239131485114292, 0.7192507562462996] and parameters: {'k': 42, 'min_k': 3, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 5, 'shrinkage': 122, 'bsl_method': 'sgd', 'bsl_n_epochs': 38, 'sgd_reg': 0.1972859643426868, 'sgd_learning_rate': 0.006655153202009401}. 
[I 2023-11-10 18:58:13,189] Trial 94 finished with values: [0.9262436545515073, 0.7240631565514828, 0.7164269721344784] and parameters: {'k': 67, 'min_k': 3, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 5, 'shrinkage': 116, 'b

In [4]:
# Define the objective function for hyperparameter optimization for KNNWithMeans
def knn_with_ZScore_objective(trial):
    # Hyperparameters to optimize
    k = trial.suggest_int('k', 20, 100)
    min_k = trial.suggest_int('min_k', 1, 10)
    sim_options = {
        'name': trial.suggest_categorical('sim_options_name', ['msd', 'cosine', 'pearson', 'pearson_baseline']),
        'user_based': trial.suggest_categorical('user_based', [False, True]),
        'min_support': trial.suggest_int('min_support', 1, 5),
    }
    # Additional parameters for pearson_baseline
    if sim_options['name'] == 'pearson_baseline':
        sim_options['shrinkage'] = trial.suggest_int('shrinkage', 0, 200)
    bsl_options = {}
    if sim_options['name'] == 'pearson_baseline':
            # Baseline options are used only with pearson_baseline similarity
        bsl_method = trial.suggest_categorical('bsl_method', ['als', 'sgd'])
        bsl_options['method'] = bsl_method
        bsl_options['n_epochs'] = trial.suggest_int('bsl_n_epochs', 10, 50)  # Default is 10
        if bsl_method == 'als':
            bsl_options['reg_u'] = trial.suggest_float('als_reg_u', 1, 20)  # Default is 15
            bsl_options['reg_i'] = trial.suggest_float('als_reg_i', 1, 20)  # Default is 10
        elif bsl_method == 'sgd':
            bsl_options['reg'] = trial.suggest_float('sgd_reg', 0.01, 0.2)  # Default is 0.02
            bsl_options['learning_rate'] = trial.suggest_float('sgd_learning_rate', 0.001, 0.01)  # Default is 0.005
    # Instantiate the KNNBasic algorithm with the suggested parameters
    algo = KNNWithZScore(k=k, min_k=min_k, sim_options=sim_options,bsl_options=bsl_options if bsl_options else None)

    # Perform cross-validation and return the mean RMSE
    # Perform cross-validation and return the mean RMSE, MAE, and FCP
    cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE', 'FCP'], cv=5, verbose=False)
    mean_rmse = np.mean(cv_results['test_rmse'])
    mean_mae = np.mean(cv_results['test_mae'])
    mean_fcp = np.mean(cv_results['test_fcp'])

    return mean_rmse, mean_mae, mean_fcp

# Create a multi-objective study object and optimize the objective function
study = optuna.create_study(directions=['minimize', 'minimize', 'maximize'], sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(knn_with_ZScore_objective, n_trials=100,n_jobs=-1)  # Number of trials can be adjusted

# Output the Pareto-optimal solutions
pareto_front = study.best_trials

# Save the Pareto-optimal solutions and their parameters to a file
with open('pareto_front_trials_and_params_KNNWithZScore.txt', 'w') as f:
    for trial in pareto_front:
        f.write(f"KNNWithZScore")
        f.write(f"Trial#{trial.number}\n")
        f.write(f"RMSE: {trial.values[0]}\n")
        f.write(f"MAE: {trial.values[1]}\n")
        f.write(f"FCP: {trial.values[2]}\n")
        f.write(f"Parameters: {trial.params}\n\n")

[I 2023-11-10 18:58:13,221] A new study created in memory with name: no-name-0242c075-c8f5-437c-9e8b-3829560dfe63


Computing the msd similarity matrix...Computing the cosine similarity matrix...

Done computing similarity matrix.
Done computing similarity matrix.
Estimating biases using als...
Estimating biases using sgd...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the cosine similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Computing the msd similarity matrix...
Comput

[I 2023-11-10 19:02:18,966] Trial 0 finished with values: [0.9531459445828787, 0.7480042523974031, 0.6954924903721141] and parameters: {'k': 25, 'min_k': 7, 'sim_options_name': 'msd', 'user_based': True, 'min_support': 4}. 


Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...Computing the pearson_baseline similarity matrix...
Computing the pearson similarity matrix...

Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson similarity matrix...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...Estimating biases using sgd...

Computing the cosine similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson similarity matrix...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


[I 2023-11-10 19:03:14,871] Trial 12 finished with values: [0.9479316200516923, 0.7396111062590588, 0.7078851922520785] and parameters: {'k': 39, 'min_k': 1, 'sim_options_name': 'pearson', 'user_based': True, 'min_support': 2}. 


...

[I 2023-11-10 19:37:34,683] Trial 92 finished with values: [0.9366221621335837, 0.7337504060238222, 0.7011276087284094] and parameters: {'k': 53, 'min_k': 9, 'sim_options_name': 'msd', 'user_based': False, 'min_support': 2}. 


Computing the pearson_baseline similarity matrix...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


[I 2023-11-10 19:37:43,157] Trial 96 finished with values: [0.9248079163428891, 0.7233801109158946, 0.7194858963326076] and parameters: {'k': 53, 'min_k': 9, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 2, 'shrinkage': 168, 'bsl_method': 'als', 'bsl_n_epochs': 36, 'als_reg_u': 1.7316805311624641, 'als_reg_i': 1.409538390197877}. 


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


[I 2023-11-10 19:37:53,113] Trial 95 finished with values: [0.9238658710132277, 0.7214199179490495, 0.721188059956927] and parameters: {'k': 26, 'min_k': 9, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 2, 'shrinkage': 170, 'bsl_method': 'als', 'bsl_n_epochs': 32, 'als_reg_u': 1.3483689920069946, 'als_reg_i': 2.3432636005676297}. 
[I 2023-11-10 19:37:54,357] Trial 97 finished with values: [0.9234088667302986, 0.7225722411020054, 0.7187803432784239] and parameters: {'k': 55, 'min_k': 9, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 2, 'shrinkage': 160, 'bsl_method': 'als', 'bsl_n_epochs': 32, 'als_reg_u': 1.1654047330399742, 'als_reg_i': 2.811284822573194}. 
[I 2023-11-10 19:37:55,287] Trial 98 finished with values: [0.9249913120065264, 0.7236857693658969, 0.7191851553953734] and parameters: {'k': 56, 'min_k': 9, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 2, 'shrinkage': 166, 'bsl_method': 'als'

In [5]:
# Define the objective function for hyperparameter optimization for KNNWithMeans
def knn_with_Baseline_objective(trial):
    # Hyperparameters to optimize
    k = trial.suggest_int('k', 20, 100)
    min_k = trial.suggest_int('min_k', 1, 10)
    sim_options = {
        'name': trial.suggest_categorical('sim_options_name', ['msd', 'cosine', 'pearson', 'pearson_baseline']),
        'user_based': trial.suggest_categorical('user_based', [False, True]),
        'min_support': trial.suggest_int('min_support', 1, 5),
    }
    # Additional parameters for pearson_baseline
    if sim_options['name'] == 'pearson_baseline':
        sim_options['shrinkage'] = trial.suggest_int('shrinkage', 0, 200)
    bsl_options = {}
    bsl_method = trial.suggest_categorical('bsl_method', ['als', 'sgd'])
    bsl_options['method'] = bsl_method
    bsl_options['n_epochs'] = trial.suggest_int('bsl_n_epochs', 10, 50)  # Default is 10
    if bsl_method == 'als':
        bsl_options['reg_u'] = trial.suggest_float('als_reg_u', 1, 20)  # Default is 15
        bsl_options['reg_i'] = trial.suggest_float('als_reg_i', 1, 20)  # Default is 10
    elif bsl_method == 'sgd':
        bsl_options['reg'] = trial.suggest_float('sgd_reg', 0.01, 0.2)  # Default is 0.02
        bsl_options['learning_rate'] = trial.suggest_float('sgd_learning_rate', 0.001, 0.01)  # Default is 0.005
    # Instantiate the KNNBasic algorithm with the suggested parameters
    algo = KNNBaseline(k=k, min_k=min_k, sim_options=sim_options,bsl_options=bsl_options if bsl_options else None)

    # Perform cross-validation and return the mean RMSE
    # Perform cross-validation and return the mean RMSE, MAE, and FCP
    cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE', 'FCP'], cv=5, verbose=False)
    mean_rmse = np.mean(cv_results['test_rmse'])
    mean_mae = np.mean(cv_results['test_mae'])
    mean_fcp = np.mean(cv_results['test_fcp'])

    return mean_rmse, mean_mae, mean_fcp

# Create a multi-objective study object and optimize the objective function
study = optuna.create_study(directions=['minimize', 'minimize', 'maximize'], sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(knn_with_Baseline_objective, n_trials=100,n_jobs=-1)  # Number of trials can be adjusted

# Output the Pareto-optimal solutions
pareto_front = study.best_trials

# Save the Pareto-optimal solutions and their parameters to a file
with open('pareto_front_trials_and_params_KNNBaseline.txt', 'w') as f:
    for trial in pareto_front:
        f.write(f"KNNWithBaseline")
        f.write(f"Trial#{trial.number}\n")
        f.write(f"RMSE: {trial.values[0]}\n")
        f.write(f"MAE: {trial.values[1]}\n")
        f.write(f"FCP: {trial.values[2]}\n")
        f.write(f"Parameters: {trial.params}\n\n")

[I 2023-11-10 19:37:57,517] A new study created in memory with name: no-name-664f15b4-a07e-4074-a9dc-859e94ed4ac8


Estimating biases using als...
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Estimating biases using als...
Computing the cosine similarity matrix...
Computing the pearson similarity matrix...
Done computing similarity matrix.Done computing similarity matrix.
Estimating biases using als...

Estimating biases using sgd...
Estimating biases using sgd...
Computing the cosine similarity matrix...
Estimating biases using als...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pearson similarity matrix...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using s

[I 2023-11-10 19:43:04,183] Trial 15 finished with values: [0.9325047732005167, 0.7361579386289767, 0.7052695166504096] and parameters: {'k': 26, 'min_k': 6, 'sim_options_name': 'cosine', 'user_based': True, 'min_support': 3, 'bsl_method': 'sgd', 'bsl_n_epochs': 21, 'sgd_reg': 0.12015325802608572, 'sgd_learning_rate': 0.007284835122899973}. 


Estimating biases using sgd...
Estimating biases using als...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases usingEstimating biases using sgd...
Computing the pearson_baseline similarity matrix...
 sgd...
Estimating biases using als...
Done computing similarity matrix.
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using als...
Computing the msd similarity matrix...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.


[I 2023-11-10 19:43:48,566] Trial 14 finished with values: [0.9322052136171285, 0.7327433796071149, 0.7092818136163613] and parameters: {'k': 29, 'min_k': 2, 'sim_options_name': 'pearson', 'user_based': True, 'min_support': 2, 'bsl_method': 'als', 'bsl_n_epochs': 25, 'als_reg_u': 11.252883533630998, 'als_reg_i': 12.08212590677828}. 


...

[I 2023-11-10 20:23:16,226] Trial 97 finished with values: [0.9262749371012035, 0.728787739948314, 0.7040008182345663] and parameters: {'k': 48, 'min_k': 6, 'sim_options_name': 'msd', 'user_based': False, 'min_support': 4, 'bsl_method': 'sgd', 'bsl_n_epochs': 19, 'sgd_reg': 0.08669557455007373, 'sgd_learning_rate': 0.006221523802675651}. 


Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.


[I 2023-11-10 20:23:36,572] Trial 98 finished with values: [0.9294592965386863, 0.7317485325342848, 0.7034056073955346] and parameters: {'k': 65, 'min_k': 6, 'sim_options_name': 'msd', 'user_based': False, 'min_support': 4, 'bsl_method': 'sgd', 'bsl_n_epochs': 21, 'sgd_reg': 0.14082707421447582, 'sgd_learning_rate': 0.00618916825118729}. 


Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.


[I 2023-11-10 20:23:41,321] Trial 99 finished with values: [0.9289905626168811, 0.7316131183728063, 0.7028769081670552] and parameters: {'k': 68, 'min_k': 5, 'sim_options_name': 'msd', 'user_based': False, 'min_support': 4, 'bsl_method': 'sgd', 'bsl_n_epochs': 32, 'sgd_reg': 0.13913835986994832, 'sgd_learning_rate': 0.005768403188097815}. 
[I 2023-11-10 20:23:43,777] Trial 96 finished with values: [0.9290359147532357, 0.7313337829811457, 0.7022030047874821] and parameters: {'k': 63, 'min_k': 2, 'sim_options_name': 'msd', 'user_based': False, 'min_support': 4, 'bsl_method': 'sgd', 'bsl_n_epochs': 19, 'sgd_reg': 0.09334657699301686, 'sgd_learning_rate': 0.0061311257708785765}. 
